# Load Data

In [ ]:
# ==== Set your local data folder here ====
DATA_DIR = "data/EEG_IMAGES"
LABELS_CSV = "data/labels.csv"
OUTPUT_DIR = "outputs/"

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)


#  1) Load labels and image metadata 

In [3]:
df_labels = pd.read_csv(LABELS_CSV)
df_labels['subject_id'] = df_labels['subject_id'].str.replace('subject_', '').astype(int)
df_labels['session'] = df_labels['session'].astype(int)
df_labels['phase'] = df_labels['phase'].astype(int)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/eeg-spectrogram-images-for-schizophrenia-detection/labels.csv'

In [ ]:
import re

def parse_filename(fname):
    match = re.match(r"subj_subject_(\d+)_s(\d+)_Phase_(\d+)\.png", fname)
    if match:
        return int(match.group(1)), int(match.group(2)), int(match.group(3))
    else:
        return None, None, None

image_files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith('.png') or f.lower().endswith('.jpg')]
df_images = pd.DataFrame({'filename': image_files})
df_images[['subject_id', 'session', 'phase']] = df_images['filename'].apply(lambda x: pd.Series(parse_filename(x)))
df_images.dropna(subset=['subject_id', 'session', 'phase'], inplace=True)
df_images['subject_id'] = df_images['subject_id'].astype(int)
df_images['session'] = df_images['session'].astype(int)
df_images['phase'] = df_images['phase'].astype(int)

In [ ]:
df_merged = pd.merge(df_images, df_labels, on=['subject_id','session','phase'], how='left')
print(f"Total images after merge: {len(df_merged)}")
print(f"Missing labels: {df_merged['label'].isnull().sum()}")

In [ ]:
if df_merged['label'].dtype != object:
    df_merged['label'] = df_merged['label'].astype(str)

# 2) Plot class distribution bar graph

In [ ]:
sns.countplot(x='label', data=df_merged, order=df_merged['label'].value_counts().index)
plt.title('Class distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'class_distribution.png'))
plt.show()

# 3) Show at least 2 sample images per class

In [ ]:
from tensorflow.keras.utils import load_img
import matplotlib.pyplot as plt
import os

# pick the correct filename column
file_col = [c for c in df_merged.columns if "filename" in c][0]

classes = df_merged['label'].unique()
plt.figure(figsize=(8, 3*len(classes)))

for i, cls in enumerate(classes):
    samples = df_merged[df_merged['label']==cls][file_col].values[:2]
    for j, fname in enumerate(samples):
        plt.subplot(len(classes), 2, i*2 + j + 1)
        img = load_img(os.path.join(DATA_DIR, fname), target_size=(224,224))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Class: {cls}", fontsize=10)

plt.suptitle("2 Sample Images per Class", fontsize=14)
plt.tight_layout(rect=[0,0,1,0.96])
plt.show()


# 4) Train/val/test split

In [ ]:
trainval_df, test_df = train_test_split(df_merged, test_size=0.20, stratify=df_merged['label'], random_state=40)
train_df, val_df = train_test_split(trainval_df, test_size=0.20, stratify=trainval_df['label'], random_state=40)

print('Train:', len(train_df), 'Val:', len(val_df), 'Test:', len(test_df))


5) ImageDataGenerators

In [ ]:
IMG_SIZE = (224,224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Use the correct filename column
file_col = [c for c in train_df.columns if "filename" in c][0]

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    directory=DATA_DIR,
    x_col=file_col,
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    directory=DATA_DIR,
    x_col=file_col,
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Important: set shuffle=False for test generator to align predictions
test_gen = test_datagen.flow_from_dataframe(
    test_df,
    directory=DATA_DIR,
    x_col=file_col,
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

NUM_CLASSES = len(train_gen.class_indices)
CLASS_INDICES = {v: k for k, v in train_gen.class_indices.items()}
print('Classes:', train_gen.class_indices)


In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import numpy as np
import seaborn as sns

In [ ]:
LR = 1e-4


#  CNN

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Paths
DATA_DIR = "/kaggle/input/eeg-spectrogram-images-for-schizophrenia-detection/EEG_IMAGES_VGG16-20250724T160523Z-1-001/EEG_IMAGES_VGG16"
LABELS_CSV = "/kaggle/input/eeg-spectrogram-images-for-schizophrenia-detection/labels.csv"

df_labels = pd.read_csv(LABELS_CSV)

# Clean subject_id column
df_labels['subject_id'] = df_labels['subject_id'].str.replace('subject_', '').astype(int)
df_labels['session'] = df_labels['session'].astype(int)
df_labels['phase'] = df_labels['phase'].astype(int)

# 2. Parse subject_id, session, phase from image filenames
def parse_filename(fname):
    # example filename: subj_subject_135_s1_Phase_4.png
    match = re.match(r"subj_subject_(\d+)_s(\d+)_Phase_(\d+)\.png", fname)
    if match:
        return int(match.group(1)), int(match.group(2)), int(match.group(3))
    else:
        return None, None, None

image_files = os.listdir(DATA_DIR)
df_images = pd.DataFrame({'filename': image_files})

df_images[['subject_id', 'session', 'phase']] = df_images['filename'].apply(
    lambda x: pd.Series(parse_filename(x))
)

# Drop images with failed parsing
df_images.dropna(subset=['subject_id', 'session', 'phase'], inplace=True)
df_images['subject_id'] = df_images['subject_id'].astype(int)
df_images['session'] = df_images['session'].astype(int)
df_images['phase'] = df_images['phase'].astype(int)

# 3. Merge image info with labels
df_merged = pd.merge(
    df_images,
    df_labels,
    on=['subject_id', 'session', 'phase'],
    how='left'
)

print(f"Total images after merge: {len(df_merged)}")
print(f"Missing labels: {df_merged['label'].isnull().sum()}")

In [ ]:
# 2. Parse subject_id, session, phase from image filenames
def parse_filename(fname):
    # example filename: subj_subject_135_s1_Phase_4.png
    match = re.match(r"subj_subject_(\d+)_s(\d+)_Phase_(\d+)\.png", fname)
    if match:
        return int(match.group(1)), int(match.group(2)), int(match.group(3))
    else:
        return None, None, None

image_files = os.listdir(DATA_DIR)
df_images = pd.DataFrame({'filename': image_files})

df_images[['subject_id', 'session', 'phase']] = df_images['filename'].apply(
    lambda x: pd.Series(parse_filename(x))
)

# Drop images with failed parsing
df_images.dropna(subset=['subject_id', 'session', 'phase'], inplace=True)
df_images['subject_id'] = df_images['subject_id'].astype(int)
df_images['session'] = df_images['session'].astype(int)
df_images['phase'] = df_images['phase'].astype(int)

# 3. Merge image info with labels
df_merged = pd.merge(
    df_images,
    df_labels,
    on=['subject_id', 'session', 'phase'],
    how='left'
)

print(f"Total images after merge: {len(df_merged)}")
print(f"Missing labels: {df_merged['label'].isnull().sum()}")


In [ ]:
# Drop rows with missing labels
df_merged.dropna(subset=['label'], inplace=True)

# Show label distribution
plt.figure(figsize=(8,5))
sns.countplot(y=df_merged['label'], order=df_merged['label'].value_counts().index)
plt.title('Class Distribution')
plt.show()

print(df_merged['label'].value_counts())


In [ ]:
# 4. Show 1 sample image per label
labels = df_merged['label'].unique()
plt.figure(figsize=(16,4))
for i, label in enumerate(labels):
    sample_img = df_merged[df_merged['label'] == label]['filename_x'].values[0]
    img = plt.imread(os.path.join(DATA_DIR, sample_img))
    plt.subplot(1, len(labels), i + 1)
    plt.imshow(img)
    plt.title(label)
    plt.axis('off')
plt.show()

In [ ]:
# 5. Train/val/test split (stratified)
trainval_df, test_df = train_test_split(df_merged, test_size=0.15, stratify=df_merged['label'], random_state=42)
train_df, val_df = train_test_split(trainval_df, test_size=0.1765, stratify=trainval_df['label'], random_state=42)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# 6. Data generators
IMG_SIZE = (224,224)
BATCH_SIZE = 32

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=DATA_DIR,
    x_col='filename_x',
    y_col='label',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

val_gen = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=DATA_DIR,
    x_col='filename_x',
    y_col='label',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_gen = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=DATA_DIR,
    x_col='filename_x',
    y_col='label',
    target_size=IMG_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# --- Model Definition ---
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# --- Define Confidence Threshold ---
CONFIDENCE_THRESHOLD = 0.8 

class HighConfidenceAccuracy(Callback):
    def __init__(self, train_gen, val_gen, threshold=0.8):
        super(HighConfidenceAccuracy, self).__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.threshold = threshold
        self.hca_train = []   
        self.hca_val = []     

    def on_epoch_end(self, epoch, logs=None):
        def get_high_conf_acc(generator):
            y_true_all = np.array(generator.classes) 
            y_pred_prob = self.model.predict(generator, verbose=0)
            y_pred = np.argmax(y_pred_prob, axis=1)
            max_probs = np.max(y_pred_prob, axis=1)
            mask = max_probs >= self.threshold

            if np.sum(mask) == 0:
                return 0.0

            y_true_filtered = y_true_all[mask]  
            y_pred_filtered = y_pred[mask]

            acc = np.mean(y_true_filtered == y_pred_filtered)
            return float(acc)

        train_hca = get_high_conf_acc(self.train_gen)
        val_hca = get_high_conf_acc(self.val_gen)

        self.hca_train.append(train_hca)
        self.hca_val.append(val_hca)

        print(f"\nEpoch {epoch + 1} - "
              f"Train Acc: {train_hca:.4f} "
              f"Val Acc: {val_hca:.4f}")

# --- Train with Fixed Callback ---
callback = HighConfidenceAccuracy(train_gen, val_gen, threshold=CONFIDENCE_THRESHOLD)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=40,
    callbacks=[callback]
)


In [ ]:
y_true_test = np.array(test_gen.classes) 
y_pred_prob_test = model.predict(test_gen, verbose=0)
y_pred_test = np.argmax(y_pred_prob_test, axis=1)
max_probs_test = np.max(y_pred_prob_test, axis=1)

mask_test = max_probs_test >= CONFIDENCE_THRESHOLD
y_true_high = y_true_test[mask_test]
y_pred_high = y_pred_test[mask_test]

if len(y_true_high) > 0:
    class_names = list(test_gen.class_indices.keys())

    # Confusion Matrix
    cm = confusion_matrix(y_true_high, y_pred_high)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix (Test - High Confidence Only)')
    plt.show()

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_true_high, y_pred_high, target_names=class_names))
else:
    print("No high-confidence predictions in test set. Lower the threshold.")

# --- Plot Training History ---
epochs = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(14, 5))

# Standard Accuracy
plt.subplot(1, 3, 1)
plt.plot(epochs, history.history['accuracy'], label='Train Accuracy')
plt.plot(epochs, history.history['val_accuracy'], label='Val Accuracy')
plt.title('Standard Accuracy')
plt.legend()


In [ ]:
model.save("CustomCNN_model.h5")


In [ ]:
from IPython.display import FileLink

# Create a link to download
FileLink("CustomCNN_model.h5")


#  VGG16 Model 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

IMG_SIZE = (224, 224)  
CONFIDENCE_THRESHOLD = 0.8

# --- Load Pretrained VGG16 Base ---
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  

# --- Build Model on Top ---
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(), 
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax') 
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# --- Custom Callback: High Confidence Accuracy ---
class HighConfidenceAccuracy(Callback):
    def __init__(self, train_gen, val_gen, threshold=0.8):
        super(HighConfidenceAccuracy, self).__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.threshold = threshold
        self.hca_train = []   
        self.hca_val = []     

    def on_epoch_end(self, epoch, logs=None):
        def get_high_conf_acc(generator):
            y_true_all = np.array(generator.classes) 
            y_pred_prob = self.model.predict(generator, verbose=0)
            y_pred = np.argmax(y_pred_prob, axis=1)
            max_probs = np.max(y_pred_prob, axis=1)
            mask = max_probs >= self.threshold

            if np.sum(mask) == 0:
                return 0.0

            y_true_filtered = y_true_all[mask]  
            y_pred_filtered = y_pred[mask]

            acc = np.mean(y_true_filtered == y_pred_filtered)
            return float(acc)

        train_hca = get_high_conf_acc(self.train_gen)
        val_hca = get_high_conf_acc(self.val_gen)

        self.hca_train.append(train_hca)
        self.hca_val.append(val_hca)

        print(f"\nEpoch {epoch + 1} - "
              f"Train HCA: {train_hca:.4f} "
              f"Val HCA: {val_hca:.4f}")

# --- Train Model ---
callback = HighConfidenceAccuracy(train_gen, val_gen, threshold=CONFIDENCE_THRESHOLD)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=40,
    callbacks=[callback],
    verbose=1
)

# --- Evaluate on Test Set ---
y_true_test = np.array(test_gen.classes)
y_pred_prob_test = model.predict(test_gen, verbose=0)
y_pred_test = np.argmax(y_pred_prob_test, axis=1)
max_probs_test = np.max(y_pred_prob_test, axis=1)

mask_test = max_probs_test >= CONFIDENCE_THRESHOLD
y_true_high = y_true_test[mask_test]
y_pred_high = y_pred_test[mask_test]

if len(y_true_high) > 0:
    class_names = list(test_gen.class_indices.keys())

    # Confusion Matrix
    cm = confusion_matrix(y_true_high, y_pred_high)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix (Test ≥ {CONFIDENCE_THRESHOLD})')
    plt.show()

    # Classification Report
    print("\n=== Classification Report ===")
    print(classification_report(y_true_high, y_pred_high, target_names=class_names))
else:
    print(f"No predictions with confidence ≥ {CONFIDENCE_THRESHOLD}. Consider lowering threshold.")

# --- Plot Training History ---
epochs = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(18, 5))

# Standard Accuracy
plt.subplot(1, 3, 1)
plt.plot(epochs, history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(epochs, history.history['val_accuracy'], label='Val Accuracy', marker='s')
plt.title('Standard Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 3, 2)
plt.plot(epochs, history.history['loss'], label='Train Loss', marker='o')
plt.plot(epochs, history.history['val_loss'], label='Val Loss', marker='s')
plt.title('Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)


plt.subplot(1, 3, 3)
plt.plot(epochs, callback.hca_train, label='Train HCA', marker='o')
plt.plot(epochs, callback.hca_val, label='Val HCA', marker='s')
plt.title(f'Accuracy (Threshold={CONFIDENCE_THRESHOLD})')
plt.xlabel('Epoch')
plt.ylabel('HCA')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# MobileNetV2

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2  # ← CHANGED
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix


IMG_SIZE = (224, 224)
CONFIDENCE_THRESHOLD = 0.6  

# --- Load Pretrained MobileNetV2 Base ---
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False 

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax') 
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


class HighConfidenceAccuracy(Callback):
    def __init__(self, train_gen, val_gen, threshold=0.6):
        super(HighConfidenceAccuracy, self).__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.threshold = threshold
        self.hca_train = []
        self.hca_val = []

    def on_epoch_end(self, epoch, logs=None):
        def get_high_conf_acc(generator):
            y_true_all = np.array(generator.classes)
            y_pred_prob = self.model.predict(generator, verbose=0)
            y_pred = np.argmax(y_pred_prob, axis=1)
            max_probs = np.max(y_pred_prob, axis=1)
            mask = max_probs >= self.threshold

            if np.sum(mask) == 0:
                return 0.0

            y_true_filtered = y_true_all[mask]
            y_pred_filtered = y_pred[mask]

            acc = np.mean(y_true_filtered == y_pred_filtered)
            return float(acc)

        train_hca = get_high_conf_acc(self.train_gen)
        val_hca = get_high_conf_acc(self.val_gen)

        self.hca_train.append(train_hca)
        self.hca_val.append(val_hca)

        print(f"\nEpoch {epoch + 1} - "
              f"Train HCA: {train_hca:.4f} "
              f"Val HCA: {val_hca:.4f}")

# --- Train Model ---
callback = HighConfidenceAccuracy(train_gen, val_gen, threshold=CONFIDENCE_THRESHOLD)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[callback],
    verbose=1
)

# --- Evaluate on Test Set 
y_true_test = np.array(test_gen.classes)
y_pred_prob_test = model.predict(test_gen, verbose=0)
y_pred_test = np.argmax(y_pred_prob_test, axis=1)
max_probs_test = np.max(y_pred_prob_test, axis=1)

# --- Diagnostic
plt.figure(figsize=(10, 4))
plt.hist(max_probs_test, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
plt.axvline(CONFIDENCE_THRESHOLD, color='red', linestyle='--', linewidth=2, label=f'Threshold = {CONFIDENCE_THRESHOLD}')
plt.title('Distribution of Maximum Prediction Probabilities (Test Set)')
plt.xlabel('Confidence (Max Softmax Probability)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n📊 Confidence Stats:")
print(f"Max confidence: {max_probs_test.max():.4f}")
print(f"Min confidence: {max_probs_test.min():.4f}")
print(f"Mean confidence: {max_probs_test.mean():.4f}")
print(f"Threshold: {CONFIDENCE_THRESHOLD}\n")

# --- Apply Threshold ---
mask_test = max_probs_test >= CONFIDENCE_THRESHOLD
y_true_high = y_true_test[mask_test]
y_pred_high = y_pred_test[mask_test]

if len(y_true_high) > 0:
    class_names = list(test_gen.class_indices.keys())

    # Confusion Matrix
    cm = confusion_matrix(y_true_high, y_pred_high)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix (Test ≥ {CONFIDENCE_THRESHOLD})')
    plt.show()

    # Classification Report
    print("\n" + "="*60)
    print("=== Classification Report ===")
    print("="*60)
    print(classification_report(y_true_high, y_pred_high, target_names=class_names))
else:
    print(f"\n❌ No predictions with confidence ≥ {CONFIDENCE_THRESHOLD}. Consider lowering threshold further (e.g., 0.5).")

# --- Plot Training History ---
epochs = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(18, 5))

# Standard Accuracy
plt.subplot(1, 3, 1)
plt.plot(epochs, history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(epochs, history.history['val_accuracy'], label='Val Accuracy', marker='s')
plt.title('Standard Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 3, 2)
plt.plot(epochs, history.history['loss'], label='Train Loss', marker='o')
plt.plot(epochs, history.history['val_loss'], label='Val Loss', marker='s')
plt.title('Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# High Confidence Accuracy
plt.subplot(1, 3, 3)
plt.plot(epochs, callback.hca_train, label='Train HCA', marker='o')
plt.plot(epochs, callback.hca_val, label='Val HCA', marker='s')
plt.title(f'High Confidence Accuracy (Threshold={CONFIDENCE_THRESHOLD})')
plt.xlabel('Epoch')
plt.ylabel('HCA')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# DenseNet121

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import DenseNet121  
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix


IMG_SIZE = (224, 224)  
CONFIDENCE_THRESHOLD = 0.5  

# --- Load Pretrained DenseNet121 Base ---
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False 

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax') 
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


class HighConfidenceAccuracy(Callback):
    def __init__(self, train_gen, val_gen, threshold=0.5):  # ← Updated default
        super(HighConfidenceAccuracy, self).__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.threshold = threshold
        self.hca_train = []
        self.hca_val = []

    def on_epoch_end(self, epoch, logs=None):
        def get_high_conf_acc(generator):
            y_true_all = np.array(generator.classes)
            y_pred_prob = self.model.predict(generator, verbose=0)
            y_pred = np.argmax(y_pred_prob, axis=1)
            max_probs = np.max(y_pred_prob, axis=1)
            mask = max_probs >= self.threshold

            if np.sum(mask) == 0:
                return 0.0

            y_true_filtered = y_true_all[mask]
            y_pred_filtered = y_pred[mask]

            acc = np.mean(y_true_filtered == y_pred_filtered)
            return float(acc)

        train_hca = get_high_conf_acc(self.train_gen)
        val_hca = get_high_conf_acc(self.val_gen)

        self.hca_train.append(train_hca)
        self.hca_val.append(val_hca)

        print(f"\nEpoch {epoch + 1} - "
              f"Train HCA: {train_hca:.4f} "
              f"Val HCA: {val_hca:.4f}")


# --- Train Model ---
callback = HighConfidenceAccuracy(train_gen, val_gen, threshold=CONFIDENCE_THRESHOLD)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[callback],
    verbose=1
)


# --- Evaluate on Test Set ---
y_true_test = np.array(test_gen.classes)
y_pred_prob_test = model.predict(test_gen, verbose=0)
y_pred_test = np.argmax(y_pred_prob_test, axis=1)
max_probs_test = np.max(y_pred_prob_test, axis=1)


# --- Diagnostic: Confidence Distribution ---
plt.figure(figsize=(10, 4))
plt.hist(max_probs_test, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
plt.axvline(CONFIDENCE_THRESHOLD, color='red', linestyle='--', linewidth=2, label=f'Threshold = {CONFIDENCE_THRESHOLD}')
plt.title('Distribution of Maximum Prediction Probabilities (Test Set)')
plt.xlabel('Confidence (Max Softmax Probability)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n📊 Confidence Stats:")
print(f"Max confidence: {max_probs_test.max():.4f}")
print(f"Min confidence: {max_probs_test.min():.4f}")
print(f"Mean confidence: {max_probs_test.mean():.4f}")
print(f"Threshold: {CONFIDENCE_THRESHOLD}\n")


# --- Apply Threshold ---
mask_test = max_probs_test >= CONFIDENCE_THRESHOLD
y_true_high = y_true_test[mask_test]
y_pred_high = y_pred_test[mask_test]

if len(y_true_high) > 0:
    class_names = list(test_gen.class_indices.keys())

    # Confusion Matrix
    cm = confusion_matrix(y_true_high, y_pred_high)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix (Test ≥ {CONFIDENCE_THRESHOLD})')
    plt.show()

    # Classification Report
    print("\n" + "="*60)
    print("=== Classification Report ===")
    print("="*60)
    print(classification_report(y_true_high, y_pred_high, target_names=class_names))
else:
    print(f"\n❌ Still no predictions with confidence ≥ {CONFIDENCE_THRESHOLD}. Consider lowering further (e.g., 0.4).")


# --- Plot Training History ---
epochs = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(18, 5))

# Standard Accuracy
plt.subplot(1, 3, 1)
plt.plot(epochs, history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(epochs, history.history['val_accuracy'], label='Val Accuracy', marker='s')
plt.title('Standard Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 3, 2)
plt.plot(epochs, history.history['loss'], label='Train Loss', marker='o')
plt.plot(epochs, history.history['val_loss'], label='Val Loss', marker='s')
plt.title('Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# High Confidence Accuracy
plt.subplot(1, 3, 3)
plt.plot(epochs, callback.hca_train, label='Train HCA', marker='o')
plt.plot(epochs, callback.hca_val, label='Val HCA', marker='s')
plt.title(f'High Confidence Accuracy (Threshold={CONFIDENCE_THRESHOLD})')
plt.xlabel('Epoch')
plt.ylabel('HCA')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# ResNet50

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50  
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix


IMG_SIZE = (224, 224)  
CONFIDENCE_THRESHOLD = 0.5  

# --- Load Pretrained ResNet50 Base ---
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False 

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax') 
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


class HighConfidenceAccuracy(Callback):
    def __init__(self, train_gen, val_gen, threshold=0.5):  
        super(HighConfidenceAccuracy, self).__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.threshold = threshold
        self.hca_train = []
        self.hca_val = []

    def on_epoch_end(self, epoch, logs=None):
        def get_high_conf_acc(generator):
            y_true_all = np.array(generator.classes)
            y_pred_prob = self.model.predict(generator, verbose=0)
            y_pred = np.argmax(y_pred_prob, axis=1)
            max_probs = np.max(y_pred_prob, axis=1)
            mask = max_probs >= self.threshold

            if np.sum(mask) == 0:
                return 0.0

            y_true_filtered = y_true_all[mask]
            y_pred_filtered = y_pred[mask]

            acc = np.mean(y_true_filtered == y_pred_filtered)
            return float(acc)

        train_hca = get_high_conf_acc(self.train_gen)
        val_hca = get_high_conf_acc(self.val_gen)

        self.hca_train.append(train_hca)
        self.hca_val.append(val_hca)

        print(f"\nEpoch {epoch + 1} - "
              f"Train HCA: {train_hca:.4f} "
              f"Val HCA: {val_hca:.4f}")


# --- Train Model ---
callback = HighConfidenceAccuracy(train_gen, val_gen, threshold=CONFIDENCE_THRESHOLD)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=40,
    callbacks=[callback],
    verbose=1
)


# --- Evaluate on Test Set ---
y_true_test = np.array(test_gen.classes)
y_pred_prob_test = model.predict(test_gen, verbose=0)
y_pred_test = np.argmax(y_pred_prob_test, axis=1)
max_probs_test = np.max(y_pred_prob_test, axis=1)


plt.figure(figsize=(10, 4))
plt.hist(max_probs_test, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
plt.axvline(CONFIDENCE_THRESHOLD, color='red', linestyle='--', linewidth=2, label=f'Threshold = {CONFIDENCE_THRESHOLD}')
plt.title('Distribution of Maximum Prediction Probabilities (Test Set)')
plt.xlabel('Confidence (Max Softmax Probability)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n📊 Confidence Stats:")
print(f"Max confidence: {max_probs_test.max():.4f}")
print(f"Min confidence: {max_probs_test.min():.4f}")
print(f"Mean confidence: {max_probs_test.mean():.4f}")
print(f"Threshold: {CONFIDENCE_THRESHOLD}\n")


# --- Apply Threshold ---
mask_test = max_probs_test >= CONFIDENCE_THRESHOLD
y_true_high = y_true_test[mask_test]
y_pred_high = y_pred_test[mask_test]

if len(y_true_high) > 0:
    class_names = list(test_gen.class_indices.keys())

    # Confusion Matrix
    cm = confusion_matrix(y_true_high, y_pred_high)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix (Test ≥ {CONFIDENCE_THRESHOLD})')
    plt.show()

    # Classification Report
    print("\n" + "="*60)
    print("=== Classification Report ===")
    print("="*60)
    print(classification_report(y_true_high, y_pred_high, target_names=class_names))
else:
    print(f"\n❌ Still no predictions with confidence ≥ {CONFIDENCE_THRESHOLD}. Consider lowering further (e.g., 0.4).")


# --- Plot Training History ---
epochs = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(18, 5))

# Standard Accuracy
plt.subplot(1, 3, 1)
plt.plot(epochs, history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(epochs, history.history['val_accuracy'], label='Val Accuracy', marker='s')
plt.title('Standard Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 3, 2)
plt.plot(epochs, history.history['loss'], label='Train Loss', marker='o')
plt.plot(epochs, history.history['val_loss'], label='Val Loss', marker='s')
plt.title('Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# High Confidence Accuracy
plt.subplot(1, 3, 3)
plt.plot(epochs, callback.hca_train, label='Train HCA', marker='o')
plt.plot(epochs, callback.hca_val, label='Val HCA', marker='s')
plt.title(f'Accuracy (Threshold={CONFIDENCE_THRESHOLD})')
plt.xlabel('Epoch')
plt.ylabel('HCA')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0  
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix


IMG_SIZE = (224, 224)  
CONFIDENCE_THRESHOLD = 0.5  


# --- Load EfficientNetB0 Base ---
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  # Freeze base

# --- Build Model ---
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_gen.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

class HighConfidenceAccuracy(Callback):
    def __init__(self, train_gen, val_gen, threshold=0.5):  
        super(HighConfidenceAccuracy, self).__init__()
        self.train_gen = train_gen
        self.val_gen = val_gen
        self.threshold = threshold
        self.hca_train = []
        self.hca_val = []

    def on_epoch_end(self, epoch, logs=None):
        def get_high_conf_acc(generator):
            y_true_all = np.array(generator.classes)
            y_pred_prob = self.model.predict(generator, verbose=0)
            y_pred = np.argmax(y_pred_prob, axis=1)
            max_probs = np.max(y_pred_prob, axis=1)
            mask = max_probs >= self.threshold

            if np.sum(mask) == 0:
                return 0.0

            y_true_filtered = y_true_all[mask]
            y_pred_filtered = y_pred[mask]

            acc = np.mean(y_true_filtered == y_pred_filtered)
            return float(acc)

        train_hca = get_high_conf_acc(self.train_gen)
        val_hca = get_high_conf_acc(self.val_gen)

        self.hca_train.append(train_hca)
        self.hca_val.append(val_hca)

        print(f"\nEpoch {epoch + 1} - "
              f"Train HCA: {train_hca:.4f} "
              f"Val HCA: {val_hca:.4f}")


# --- Train Model ---
callback = HighConfidenceAccuracy(train_gen, val_gen, threshold=CONFIDENCE_THRESHOLD)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=40,
    callbacks=[callback],
    verbose=1
)


# --- Evaluate on Test Set ---
y_true_test = np.array(test_gen.classes)
y_pred_prob_test = model.predict(test_gen, verbose=0)
y_pred_test = np.argmax(y_pred_prob_test, axis=1)
max_probs_test = np.max(y_pred_prob_test, axis=1)


plt.figure(figsize=(10, 4))
plt.hist(max_probs_test, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
plt.axvline(CONFIDENCE_THRESHOLD, color='red', linestyle='--', linewidth=2, label=f'Threshold = {CONFIDENCE_THRESHOLD}')
plt.title('Distribution of Maximum Prediction Probabilities (Test Set)')
plt.xlabel('Confidence (Max Softmax Probability)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n📊 Confidence Stats:")
print(f"Max confidence: {max_probs_test.max():.4f}")
print(f"Min confidence: {max_probs_test.min():.4f}")
print(f"Mean confidence: {max_probs_test.mean():.4f}")
print(f"Threshold: {CONFIDENCE_THRESHOLD}\n")


# --- Apply Threshold ---
mask_test = max_probs_test >= CONFIDENCE_THRESHOLD
y_true_high = y_true_test[mask_test]
y_pred_high = y_pred_test[mask_test]

if len(y_true_high) > 0:
    class_names = list(test_gen.class_indices.keys())

    # Confusion Matrix
    cm = confusion_matrix(y_true_high, y_pred_high)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix (Test ≥ {CONFIDENCE_THRESHOLD})')
    plt.show()

    # Classification Report
    print("\n" + "="*60)
    print("=== Classification Report ===")
    print("="*60)
    print(classification_report(y_true_high, y_pred_high, target_names=class_names))
else:
    print(f"\n❌ Still no predictions with confidence ≥ {CONFIDENCE_THRESHOLD}. Consider lowering further (e.g., 0.4).")


# --- Plot Training History ---
epochs = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(18, 5))

# Standard Accuracy
plt.subplot(1, 3, 1)
plt.plot(epochs, history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(epochs, history.history['val_accuracy'], label='Val Accuracy', marker='s')
plt.title('Standard Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 3, 2)
plt.plot(epochs, history.history['loss'], label='Train Loss', marker='o')
plt.plot(epochs, history.history['val_loss'], label='Val Loss', marker='s')
plt.title('Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# High Confidence Accuracy
plt.subplot(1, 3, 3)
plt.plot(epochs, callback.hca_train, label='Train HCA', marker='o')
plt.plot(epochs, callback.hca_val, label='Val HCA', marker='s')
plt.title(f'Accuracy (Threshold={CONFIDENCE_THRESHOLD})')
plt.xlabel('Epoch')
plt.ylabel('HCA')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Comparision

In [ ]:
import matplotlib.pyplot as plt

# Your accuracy data
models = ['CNN', 'VGG16', 'MobileNet', 'DenseNet', 'ResNet50']
accuracy = [0.90, 0.97, 0.82, 0.79, 0.52]

# Create plot
plt.figure(figsize=(10, 6))
bars = plt.bar(models, accuracy, color=['#4CAF50', '#2196F3', '#FF9800', '#9C27B0', '#F44336'],
               edgecolor='black', linewidth=1.2)

# Annotate bars with accuracy values
for bar, acc in zip(bars, accuracy):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01,
             f'{acc:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=12)

# Customize
plt.title('📊 Model Accuracy Comparison', fontsize=16, fontweight='bold')
plt.ylabel('Accuracy', fontsize=14)
plt.ylim(0, 1.05)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Show plot
plt.tight_layout()
plt.show()